In [ ]:
import os
import json
import faiss
import numpy as np
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

/home/cecilio/Projetos/meioGPT/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
load_dotenv()

API_KEY = os.getenv("OPENROUTER_KEY")
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=API_KEY,
)

model = SentenceTransformer('modelo_podrao')
index = faiss.read_index("faiss_index.index")

instructions = os.getenv("INSTRUCTIONS")

with open('conversas_podroes_filtrado.json', 'r', encoding='utf-8') as f:
    conversas = json.load(f)

In [3]:
def buscar_contexto(mensagem, k=5):
    # Gerar embedding para a mensagem
    query_embedding = model.encode([mensagem], convert_to_tensor=False)
    query_embedding = np.array(query_embedding).astype("float32")
    
    # Buscar os k vetores mais próximos no índice FAISS
    distances, indices = index.search(query_embedding, k)
    
    # Recuperar as mensagens de contexto com base nos índices
    context_msgs = [conversas[i]["message"] for i in indices[0]]
    return "\n".join(context_msgs)

In [4]:
query = "Filme chato pra caralho"
contexto = buscar_contexto(query)
print("Resposta gerada:", contexto)

Resposta gerada: Caralho... esse filme é muito trash e muito velho!!!
Burro pra caralho
Estou no cinema pra assistir duna 2, devo jogar mais pra noite
Esse filme marcou minha infância
O filme é uma piada com os estúdios que forçaram a barra para acontecer.


In [5]:
def chat_whatsapp(mensagem):
    contexto = buscar_contexto(mensagem)

    prompt = f"""
    {instructions}
    Aqui estão algumas mensagens anteriores para basear em sua resposta:
    {contexto}

    Agora responda a esta nova mensagem: "{mensagem}"
    """
    
    resposta = client.chat.completions.create(
        model="openai/gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    return resposta

mensagem = "Conseguiu fazer o servidor de Valheim rodar?"
bot_resposta = chat_whatsapp(mensagem)
resposta = bot_resposta.choices[0].message.content.strip()
print(resposta)

Ah, claro, fiz um ritual com velas e tudo! O servidor do Valheim agora está mais ativo que eu tentando entender as piadas do tio do pavê. Mas, para falar a verdade, ele estava tão parado que até pensei em colocar uma placa de "em breve" na frente. Mas tudo bem, ao menos a tecla F5 já fez amizade com o botão "iniciar" e eles estão rodando felizes agora!
